# Data-Driven real-estate investment risk model
## Cleaning-Combine data

First step in this project is to clean and combine data. This data is then used for models.

I focus on 6 factors affecting Market risk and **Change in quarterly returns**. Below is the list of all 6 factors.

## Factors
1. Market Occupancy
\begin{equation*}
\: Market \: Occupancy = \:1- Vacancy \:Rate
\end{equation*}
2. Housing demand quotient 
\begin{equation*}
Housing \: demand \: quotient= \frac{Total \: jobs \: gained \: in \: last \: 12 \: months * House \: ownership \: rate}{Total \: vacant \: units \: + \: Total \: units \: under \: construction}
\end{equation*}
    * Total jobs gained in 12 months
    * Number of vacant units
    * Number of units under construction
3. Total % of Market under construction
4. Number of  Employed
5. Unemployment Rate
6. Population Estimate

Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

Although it is not advisable but to keep this notebook clean and short, supress warnings.

Please, comment this when you want to see warnings

In [2]:
import warnings
warnings.filterwarnings("ignore")

## Loading data

I am looking at the markets of 14 Metropolitan areas across USA. These areas are identified by their MSA (Metropolitan Statistical Area) codes.

Importing msa codes data.  

In [3]:
MSA_codes=pd.read_excel(r'Data/MSA_codes.xlsx')

For these markets I need Home ownership rate to calculate housing demand quotient. 

Importing Home ownership rate data.

In [4]:
HomeOwnershipRate=pd.read_excel(r'Data/HomeOwnershipRate.xlsx')

Combining these to files into one.

In [5]:
Markets=HomeOwnershipRate.merge(MSA_codes,on='Market')

Here is the list of All Markets with their msa code and Home-ownership rates.

In [6]:
Markets

,Market,Homeownership Rate,msa code,Metropolitan area,State
0,Atlanta,43.8,12060,Atlanta‐Sandy Springs‐Roswell,GA
1,Boston,35.2,14454,Boston,MA
2,Charlotte,53.2,16740,Charlotte‐Concord‐Gastonia,NC
3,Chicago,44.9,16974,Chicago‐Naperville‐Arlington Heights,IL
4,Dallas,39.4,19124,Dallas-Plano-Irving,TX
5,Denver,50.0,19740,Denver-Aurora-Lakewood,CO
6,Houston,42.8,26420,Houston-The Wood Lands-Sugar Land,TX
7,Kansas_City,64.6,28140,Kansas City,KS
8,Los_Angeles,48.4,31084,Los Angeles-Long Beach-Glendale,CA
9,Philadelphia,66.1,37964,Philadelphia,PA


To calculate change in returns I used NCREIF property index to get data on total returns. This step will few seconds.

In [7]:
NPI_MSA=pd.read_excel(r'Data/NPI Annualized MSA_20191.xlsx')
NPI_MSA.head()

,iname,type,region,division,state,msa code,yyq,COUNT,emv,bmv,...,TotalReturnsYear4,TotalReturnsYear5,TotalReturnsYear6,TotalReturnsYear7,TotalReturnsYear8,TotalReturnsYear9,TotalReturnsYear10,TotalReturnsYear15,TotalReturnsYear20,InceptionTotalReturns
0,CO_M_10900,-,E,NE,PA,10900,20064,7,197481659,195877041,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.745402
1,CO_M_10900,-,E,NE,PA,10900,20071,8,214246637,210016225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.365249
2,CO_M_10900,-,E,NE,PA,10900,20072,8,142740000,214246637,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.081602
3,CO_M_10900,-,E,NE,PA,10900,20073,8,143985150,142740000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.597933
4,CO_M_10900,-,E,NE,PA,10900,20074,8,145133988,143985150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.194026


I dropped columns with data not relavant to current task and focus on following features

In [8]:
NPI_features=['type','state','yyq','msa code','TotalReturn']

['type', 'state', 'yyq', 'msa code', 'TotalReturn']


In [9]:
NPI_MSA=NPI_MSA[NPI_features]

List of MSA (Metropolitan Statistical Area) codes. I join the data on factors to NCREIF property index data based on these codes so to add a new market data make sure the it's MSA code is present in this list.

In [10]:
print('List of MSA codes in the NCREIF property index data')
print(NPI_MSA['msa code'].unique())

[10900 11244 12060 12420 12580 12940 13820 14454 14500 14860 15764 15804
 15980 16700 16740 16974 17140 17820 17900 18140 19124 19740 19804 20500
 20994 21660 22744 23104 24860 25420 25540 26420 26900 27260 27600 28140
 28940 29404 29460 29820 30140 31084 31140 32820 33124 33340 33460 33874
 34940 34980 35004 35084 35300 35614 35840 36084 36420 36740 37100 37964
 38060 38300 38900 39300 39580 39900 40060 40140 40484 40900 41180 41620
 41700 41740 41884 41940 42034 42200 42220 42340 42644 43524 44700 45104
 45300 45940 46060 46520 46700 47260 47664 47894 48424 48864 48900 49340]


I created a function to calculate change in quarterly returns for each of the markets using NCREIF data. In the end I use this function again for all the markets. 

In [11]:
def ChangeInReturns(msa_code,Ncrief_data):
    # selecting Ncrief_data based on particular MSA code and property type 'Apartments' ('A')
    Returns_Data=Ncrief_data[(Ncrief_data['msa code']==msa_code)&(Ncrief_data.type=='A')]
    # extract informatation on year and quarter using 'yyq' column
    # where first 4 letters represent the year and last represent the quarter
    Returns_Data['Year']=(Returns_Data.yyq/10).astype(int)
    Returns_Data['Quarter']=Returns_Data.yyq%10
    # create 'Date' column using year and quarter data, date format YYYY-MM-01
    Returns_Data['Date']=pd.to_datetime(pd.DataFrame({'year': Returns_Data.Year,
                                                      'month': 3*Returns_Data.Quarter-2,
                                                      'day':np.ones(len(Returns_Data))}))
    # Calculate change in quarterly returns from total returns data
    Del_returns=Returns_Data['TotalReturn'].diff()
    #Adjusting datatable and adding 'Change in returns' column to it
    Returns_Data=Returns_Data[1:]
    Returns_Data['Change In Return']=Del_returns
    #dropping features no longer needed in next steps
    Returns_Data.drop(['type','yyq','TotalReturn','Year','Quarter'],axis=1,inplace=True)
    return Returns_Data

As an example, I call this function with inputs as msa code for Seattle and total returns data.

In [12]:
ChangeInReturns(Markets.loc[13,'msa code'],NPI_MSA).head()

,state,msa code,Date,Change In Return
38920,WA,42644,1989-01-01,-0.142581
38921,WA,42644,1989-04-01,-0.493043
38922,WA,42644,1989-07-01,0.243908
38923,WA,42644,1989-10-01,3.182987
38924,WA,42644,1990-01-01,-3.201320


To get data for different factors I used data collected for each of the market saved in separate folders. 
Starting with datafile with name ending with 'CoStar', I get data on **market occupancy**, **number of vacant units**, **number of units under construction**, **total % of market under construction**. All data is collected at quarterly basis.

In [13]:
def Factors_Data(City):
    # Set up the file-path for each Market
    path_string=r'Data/Market/' + City + '/' + City
    # read data from '_Costar' datafile
    DataTable=pd.read_excel(path_string+'_CoStar.xlsx')
    # Clean up any row that with non numeric 'Period' data
    DataTable=DataTable[DataTable.Period.map(lambda x:x[0])=='2']
    # extract informatation on year and quarter using 'Period' feature to create a column for 'Date'
    # Date is in format YYYY-MM-01
    # where first 4 letters represent the year and last represent the quarter
    DataTable['Year']=DataTable.Period.map(lambda x:x[:4])
    DataTable['Quarter']=DataTable.Period.map(lambda x:x[6:])
    DataTable['Date']=pd.to_datetime(pd.DataFrame({'year': DataTable.Year.astype(int),
                                                   'month': 3*DataTable.Quarter.astype(int)-2,
                                                   'day':np.ones(len(DataTable))}))
    # drop the 'Period','Year','Quarter' columns after creating 'Date' column
    DataTable.drop(['Period','Year','Quarter'],axis=1,inplace=True)
    # Create column on Market occupancy from vacancy rate column (vacancy rate is between 0 to 1, not in percents)
    DataTable['Market Occupancy']=1-DataTable['Vacancy Rate']
    # Create column for Number of vacant units using 'Inventory Units' and 'Vacancy Rate'
    DataTable['Vacant_units']=DataTable['Inventory Units'].multiply(DataTable['Vacancy Rate']).astype(int)
    # Finally drop all features not need
    DataTable.drop(['Asset Value','Market Asking Rent/Unit','Market Cap Rate','12 Month Absorption Units',
                    'Market Asking Rent/Unit Growth','Market Asking Rent/Unit Growth','Vacancy Rate','Inventory Units',
                    'Market Sale Price/Unit','12 Month Sales Volume','12 Month Sales Vol Growth'],axis=1,inplace=True)
    # The final datatable has columns on  number of units under construction and total % of market under construction
    # these do not need any feature engineering and are taken as is
    return DataTable

As an example, I call this function for the market of Seattle.

In [14]:
Factors_Data(Markets.Market[13]).tail()

,Under Constr Units,Under Constr % of Inventory,Date,Market Occupancy,Vacant_units
93,6629.0,0.028742,2001-01-01,0.944235,12861
94,6918.0,0.030104,2000-10-01,0.947673,12024
95,6370.0,0.027791,2000-07-01,0.949598,11552
96,7986.0,0.035268,2000-04-01,0.948913,11567
97,9359.0,0.041744,2000-01-01,0.948977,11439


Next, I used the datafile ending with '_Employment' to get data on 'Total Number of Employed' and 'Total jobs gained in 12 months' for each Market. This data is collected on monthly basis.

In [15]:
def Emp_Data(City):
    # Set up the file-path for each Market
    path_string=r'Data/Market/' + City + '/' + City
    # read data from '_Employment' datafile
    Employment_Data=pd.read_excel(path_string+ '_Employment.xlsx',skiprows=12,
                              usecols=["Year", "Period", "Label", "Observation Value"])
    # create 'Date' column using 'year' and 'month' columns. the format of Date is YYYY-MM-01
    Employment_Data['Date']=pd.to_datetime(pd.DataFrame({'year': Employment_Data.Year,
                                                      'month': Employment_Data.Period.map(lambda x: x[-2:]).astype(int),
                                                      'day':np.ones(len(Employment_Data))}))
    # Create column for 'Total Employed' as the employment data is given in order of 1000.
    Employment_Data['Total Employed']=Employment_Data['Observation Value']*1000
    # Estimate job gain in 1 month for Total Employed
    Employment_Data['Job_Gain_1_month']=Employment_Data['Total Employed'].diff()
    Employment_Data.fillna(0,inplace=True)
    # Estimate job gain in 12 month from jobs gained each month data.
    Employment_Data['Job_Gain_12_month']=Employment_Data['Job_Gain_1_month'].rolling(12).sum()
    # drop columns not neccessary in modeling
    Employment_Data.drop(['Year','Period','Label','Observation Value'],axis=1,inplace=True)
    return Employment_Data

As an example, I call this function for the market of Seattle.

In [16]:
Emp_Data(Markets.Market[1]).tail()

,Date,Total Employed,Job_Gain_1_month,Job_Gain_12_month
347,2018-12-01,2771100.0,-2900.0,15800.0
348,2019-01-01,2781900.0,10800.0,20700.0
349,2019-02-01,2787400.0,5500.0,14900.0
350,2019-03-01,2789500.0,2100.0,20000.0
351,2019-04-01,2791900.0,2400.0,25800.0


Next, I used the datafile ending with '_Unemployment' to get data on unemployment rate for each Market. This data is collected on monthly basis.

In [17]:
def UnEmp_Data(City):
    # Set up the file-path for each Market
    path_string=r'Data/Market/' + City + '/' + City
    # read data from '_Unemployment' datafile
    Unemployment_Data=pd.read_excel(path_string + '_Unemployment.xlsx',skiprows=11,
                              usecols=["Year", "Period", "Label", "Observation Value"])
    # create 'Date' column using 'year' and 'month' columns. the format of Date is YYYY-MM-01
    Unemployment_Data['Date']=pd.to_datetime(pd.DataFrame({'year': Unemployment_Data.Year,
                                                          'month': Unemployment_Data.Period.map(lambda x: x[-2:]).astype(int),
                                                          'day':np.ones(len(Unemployment_Data))}))
    # renaming the column 'Observation Value' to 'Unemployment Rate'
    Unemployment_Data.rename(columns={'Observation Value':'Unemployment Rate'},inplace=True)
    # dropping non-necessary columns
    Unemployment_Data.drop(['Year','Period','Label'],axis=1,inplace=True)
    return Unemployment_Data

First few rows of Monthly Unemployment rate data for Seattle market.

In [18]:
UnEmp_Data(Markets.Market[13]).tail()

,Unemployment Rate,Date
347,4.1,2018-12-01
348,4.4,2019-01-01
349,4.4,2019-02-01
350,4.4,2019-03-01
351,3.7,2019-04-01


Final component of the model is Population, for which I looked on the Yearly Population Estimates for each of the (Metropolitan Statistical Area). 

I got the data from following link to the 'ALFRED archival economic data'

https://alfred.stlouisfed.org/release?et=&ob=t&od=&pageID=2&rd=&rid=111&t=

I searched for each of the MSA and downloaded the data for it.

In [19]:
def Market_PopEst(City):
    # Set up the file-path for each Market
    path_string=r'Data/MSAwise population estimates/'+City+'_PopEst.csv'
    # read data from '_PopEst' datafile saved in separate folder named 'MSAwise population estimates' 
    PopEst=pd.read_csv(path_string)
    # convert data in 'Date' column from string format to date format
    PopEst['DATE']=pd.to_datetime(PopEst.DATE)
    # set the name of the columns
    PopEst.columns=['Date','Population Estimate']
    # Population Estimate is provided in order of 1000 so change it to give actual value
    PopEst['Population Estimate']=PopEst['Population Estimate']*1000
    # Add name of the Market to each population estimate
    PopEst['Market']=[City for i in range(len(PopEst))]
    return PopEst

Seattle yearly population estimates.

In [20]:
Market_PopEst(Markets.Market[13])

,Date,Population Estimate,Market
0,2000-01-01,3052379.0,Seattle
1,2001-01-01,3094380.0,Seattle
2,2002-01-01,3121895.0,Seattle
3,2003-01-01,3138938.0,Seattle
4,2004-01-01,3163703.0,Seattle
5,2005-01-01,3202388.0,Seattle
6,2006-01-01,3259945.0,Seattle
7,2007-01-01,3307360.0,Seattle
8,2008-01-01,3356637.0,Seattle
9,2009-01-01,3407848.0,Seattle


After creating functions to extract the data for different factors and change in quarterly return I come to the most important step
Finally, I call the functions I created above one by one for each market. I combined all the data in one spreadsheet.

In [21]:
print('Adding data for Markets:')
for i in range(len(Markets)): # run a loop over Markets
    print(Markets.Market[i]) # print which market data we are adding
    # run the first function to get data on 
    # 'market occupancy', 'number of vacant units', 'number of units under construction', 'total % of market under construction'
    DataTable=Factors_Data(Markets.Market[i]) 
    # run the second function to get data on 'Total Number of Employed' and 'Total jobs gained in 12 months'
    Employment_Data=Emp_Data(Markets.Market[i])
    # run the third function to get data on 'Unemployment rate'
    Unemployment_Data=UnEmp_Data(Markets.Market[i])
    # First I join 'DataTable' and 'Employment_Data' using 'Date' so that data with same date from both tables comes in one row
    #I make sure that data is quarterly not monthly by doing an inner-join on the tables.
    Data=DataTable.merge(Employment_Data,on=['Date'],how='inner')
    # Next I join 'Unemployment_Data' to this new combined table using 'Date' again and is is quarterly not monthly
    Data=Data.merge(Unemployment_Data,on=['Date'],how='inner')
    # I restricted my model to take data from Jan-2000 to 2018 end, because both tables have common data during this time
    # If more data becomes available for larger time period, changes should be made accordingly
    Data=Data[(Data['Date']>='2000-01-01')&(Data['Date']<'2019-01-01')]
    #One missing factor not calculated till now is Housing Demand Quotient, it required data from two different table
    #after combining the tables I calculate 'Housing Demand Quotient' in a new column added to the combined table.
    Data['Housing Demand Quotient']=Markets.loc[i,'Homeownership Rate']*Data['Job_Gain_12_month']/(Data['Vacant_units']+Data['Under Constr Units'])
    # drop columns not needed in modeling.
    Data.drop(['Under Constr Units','Vacant_units','Job_Gain_1_month','Job_Gain_12_month'],axis=1,inplace=True)
    # Next I join the Change in quarterly returns data to the combined table,  using 'Date' so that data with same date
    # from both tables comes in one row
    Data=Data.merge(ChangeInReturns(Markets.loc[i,'msa code'],NPI_MSA),on=['Date'],how='inner')
    # Last piece of the puzzle is 'Population Estimate' data. I join it using the date, so same date data 
    # from both tables comes in one row, but in this case I use left-join so that data is quarterly rather than yearly
    Data=Data.merge(Market_PopEst(Markets.Market[i]),on=['Date'],how='left')
    # Since the population estimate is on yearly basis, to make data into quarterly I performed linear interpolation
    Data['Population Estimate']=Data['Population Estimate'].interpolate(method='linear',limit_direction='both')
    Data.Market.fillna(Markets.Market[i],inplace=True)
    # Create/Add to main datatable
    if i==0:
        FinalData=Data
    else:
        FinalData=Data.append(FinalData)

Adding data for Markets:
Atlanta
Boston
Charlotte
Chicago
Dallas
Denver
Houston
Kansas_City
Los_Angeles
Philadelphia
Phoenix
Portland
San_Francisco
Seattle


First few rows of clean data looks like follows,

In [22]:
FinalData.head()

,Under Constr % of Inventory,Date,Market Occupancy,Total Employed,Unemployment Rate,Housing Demand Quotient,state,msa code,Change In Return,Population Estimate,Market
0,0.077813,2018-10-01,0.941469,1736200.0,3.8,44.477334,WA,42644,-0.207388,3939363.0,Seattle
1,0.075314,2018-07-01,0.944947,1724400.0,4.0,40.722087,WA,42644,0.340640,3939363.0,Seattle
2,0.077475,2018-04-01,0.945757,1716400.0,3.7,42.028420,WA,42644,0.628392,3939363.0,Seattle
3,0.074218,2018-01-01,0.943161,1710900.0,4.6,50.819533,WA,42644,-1.976761,3939363.0,Seattle
4,0.073655,2017-10-01,0.940336,1694600.0,4.0,43.767138,WA,42644,-0.485984,3925639.5,Seattle


Saving cleaned data to a csv file. This csv file will be used for data visualization, model training, running the app-file.

In [23]:
FinalData.to_csv('Input_data_market_risk_model.csv',index=False)

Done with data, proceed to model training or launching app.